## Numerical Analysis - Fall semester 2024

# Serie 05 - Interpolation, splines, and least squares fitting

Import of the NumPy and matplotlib packages. This week, we will also need the SciPy package, an extension of the NumPy package for some more advanced scientific computations.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp

<hr style="clear:both">

### Stability of interpolants

We are going to study the stability of the Lagrange's interpolation polynomial on equally distributed nodes, on Clenshaw-Curtis's nodes and the piecewise linear interpolation polynomial. We consider the function

$$
f(x) = \sin(x) + x,
$$

which we interpolate in the interval $[0,10]$ at $n+1$ nodes $x_i$, $i=1,...,n+1$. We consider two sets of data $(x_i,y_i)$ where $y_i=f(x_i)$ and
$(x_i,z_i)$ where and $z_i$ is a perturbation of $y_i$ given by $z_i=y_i+\varepsilon_i$ with $\varepsilon_i$ a uniform random variable in $(-0.1,0.1)$. Such a perturbation can, for example, be present due to measurement errors. Once the nodes $x$, the function $f$, and the degree $n$ are defined, you may use the below function to generate the data.

In [ ]:
def get_data(x, f, n, seed=None):
    y = f(x)
    np.random.seed(seed)
    e = np.random.uniform(low=-0.1, high=0.1, size=n+1)
    z = y + e
    return y, z

<div class="alert alert-warning">
    
**Warning:** Due to the random perturbation, the data will change very time you call this function. To avoid this from happening, you fix set a seed (any integer of your choice, often `0`) by passing it as the last argument of the `get_data` function, e.g. `get_data(..., seed=0)`.
</div>

<div class="alert alert-success">
    
**Exercise 1:** Use $n+1$ uniformly distributed nodes to compute the degree $n$ least-squares polynomial obtained for the two data sets. Plot the polynomials along with the function $f$ by evaluating each of them at $100$ uniformly spaced points on the interval $[0, 10]$. Do this once for $n=4$ and once for $n=15$. What can we observe?

*Hint:* Make use the `np.polyfit` and `np.polyval` functions, as you have learned in last week's notebook.

</div>

In [ ]:
### BEGIN SOLUTION

def f(x):
    return np.sin(x) + x

x_lin = np.linspace(0, 10, 100)  # fine grid

x_4_unif = np.linspace(0, 10, 4 + 1)
y_4_unif, z_4_unif = get_data(x_4_unif, f, 4)
p_4_y_unif = np.polyval(np.polyfit(x_4_unif, y_4_unif, 4), x_lin)
p_4_z_unif = np.polyval(np.polyfit(x_4_unif, z_4_unif, 4), x_lin)

plt.scatter(x_4_unif, y_4_unif, label=r"$(x_i, y_i)$")
plt.plot(x_lin, p_4_y_unif, label=r"interpolant $p_4(x)$ of $(x_i, y_i)$")
plt.scatter(x_4_unif, z_4_unif, label=r"$(x_i, z_i)$")
plt.plot(x_lin, p_4_z_unif, label=r"interpolant $p_4(x)$ of $(x_i, z_i)$")
plt.plot(x_lin, f(x_lin), label=r"function $f(x)$")
plt.grid()
plt.ylabel(r"$y$")
plt.xlabel(r"$x$")
plt.legend()
plt.show()

x_15_unif = np.linspace(0, 10, 15 + 1)
y_15_unif, z_15_unif = get_data(x_15_unif, f, 15)
p_15_y_unif = np.polyval(np.polyfit(x_15_unif, y_15_unif, 15), x_lin)
p_15_z_unif = np.polyval(np.polyfit(x_15_unif, z_15_unif, 15), x_lin)

plt.scatter(x_15_unif, y_15_unif, label=r"$(x_i, y_i)$")
plt.plot(x_lin, p_15_y_unif, label=r"interpolant $p_{15}(x)$ of $(x_i, y_i)$")
plt.scatter(x_15_unif, z_15_unif, label=r"$(x_i, z_i)$")
plt.plot(x_lin, p_15_z_unif, label=r"interpolant $p_{15}(x)$ of $(x_i, z_i)$")
plt.plot(x_lin, f(x_lin), label=r"function $f(x)$")
plt.grid()
plt.ylabel(r"$y$")
plt.xlabel(r"$x$")
plt.legend()
plt.show()

# Let us first remark that for this function $f$, the interpolating polynomial for the
# unperturbed data converges to the function (better approximation in the case $n=15$
# than in the case $n=4$). In the case $n=15$, the interpolating polynomial based on
# the perturbed data clearly oscillates at the extremities of the interval. Small
# perturbations on the data (less than $10$%) have in fact made a big difference between
# the two interpolating polynomials at the ends of the interval. The interpolating polynomial
# is therefore not stable. This is due to the fact that the Lebesgue's constant $L_n$ sharply
# increases when $n$ increases. This phenomenon would be amplified if we increased the value of $n$.

### END SOLUTION

<div class="alert alert-success">
    
**Exercise 2:** Repeat Exercise 1, but instead of uniformly spaced nodes, use the Clenshaw-Curtis nodes. What can you observe?

*Hint:* You can copy all of your code from Exercise 1. The only thing you will need to change is the definition of the nodes.
</div>

In [ ]:
### BEGIN SOLUTION

x_4_cc = (10 + 0) / 2 + (10 - 0) / 2 * np.cos(np.pi * np.arange(4 + 1) / 4) 
y_4_cc, z_4_cc = get_data(x_4_cc, f, 4)
p_4_y_cc = np.polyval(np.polyfit(x_4_cc, y_4_cc, 4), x_lin)
p_4_z_cc = np.polyval(np.polyfit(x_4_cc, z_4_cc, 4), x_lin)

plt.scatter(x_4_cc, y_4_cc, label=r"$(x_i, y_i)$")
plt.plot(x_lin, p_4_y_cc, label=r"interpolant $p_4(x)$ of $(x_i, y_i)$")
plt.scatter(x_4_cc, z_4_cc, label=r"$(x_i, z_i)$")
plt.plot(x_lin, p_4_z_cc, label=r"interpolant $p_4(x)$ of $(x_i, z_i)$")
plt.plot(x_lin, f(x_lin), label=r"function $f(x)$")
plt.grid()
plt.ylabel(r"$y$")
plt.xlabel(r"$x$")
plt.legend()
plt.show()

x_15_cc = (10 + 0) / 2 + (10 - 0) / 2 * np.cos(np.pi * np.arange(15 + 1) / 15) 
y_15_cc, z_15_cc = get_data(x_15_cc, f, 15)
p_15_y_cc = np.polyval(np.polyfit(x_15_cc, y_15_cc, 15), x_lin)
p_15_z_cc = np.polyval(np.polyfit(x_15_cc, z_15_cc, 15), x_lin)

plt.scatter(x_15_cc, y_15_cc, label=r"$(x_i, y_i)$")
plt.plot(x_lin, p_15_y_cc, label=r"interpolant $p_{15}(x)$ of $(x_i, y_i)$")
plt.scatter(x_15_cc, z_15_cc, label=r"$(x_i, z_i)$")
plt.plot(x_lin, p_15_z_cc, label=r"interpolant $p_{15}(x)$ of $(x_i, z_i)$")
plt.plot(x_lin, f(x_lin), label=r"function $f(x)$")
plt.grid()
plt.ylabel(r"$y$")
plt.xlabel(r"$x$")
plt.legend()
plt.show()

# In this case, the perturbations on the data have produced only small modifications on
# the polynomial interpolation. We can explain this by using the fact that for the
# Clenshaw-Curtis nodes, the Lebesgue constant grows slowler than for equidistant nodes.

### END SOLUTION

<div class="alert alert-success">
    
**Exercise 3:** Repeat Exercise 1, but instead of the least-squares use a piecewise linear interpolant. Comment on the result.

*Hint:* You can reuse most of the code from Exercise 1. The function `np.interp(x_eval, x, y)` which takes as input a list `x_eval` of points and some data `x`,`y` evaluates the piecewise linear interpolant of the data in the points `x_eval`.
</div>

In [ ]:
### BEGIN SOLUTION

x_4_pw = np.linspace(0, 10, 4 + 1)
y_4_pw, z_4_pw = get_data(x_4_pw, f, 4)
p_4_y_pw = np.interp(x_lin, x_4_pw, y_4_pw)
p_4_z_pw = np.interp(x_lin, x_4_pw, z_4_pw)

plt.scatter(x_4_pw, y_4_pw, label=r"$(x_i, y_i)$")
plt.plot(x_lin, p_4_y_pw, label=r"interpolant $p_4(x)$ of $(x_i, y_i)$")
plt.scatter(x_4_pw, z_4_pw, label=r"$(x_i, z_i)$")
plt.plot(x_lin, p_4_z_pw, label=r"interpolant $p_4(x)$ of $(x_i, z_i)$")
plt.plot(x_lin, f(x_lin), label=r"function $f(x)$")
plt.grid()
plt.ylabel(r"$y$")
plt.xlabel(r"$x$")
plt.legend()
plt.show()

x_15_pw = np.linspace(0, 10, 15 + 1)
y_15_pw, z_15_pw = get_data(x_15_pw, f, 15)
p_15_y_pw = np.interp(x_lin, x_15_pw, y_15_pw)
p_15_z_pw = np.interp(x_lin, x_15_pw, z_15_pw)

plt.scatter(x_15_pw, y_15_pw, label=r"$(x_i, y_i)$")
plt.plot(x_lin, p_15_y_pw, label=r"interpolant $p_{15}(x)$ of $(x_i, y_i)$")
plt.scatter(x_15_pw, z_15_pw, label=r"$(x_i, z_i)$")
plt.plot(x_lin, p_15_z_pw, label=r"interpolant $p_{15}(x)$ of $(x_i, z_i)$")
plt.plot(x_lin, f(x_lin), label=r"function $f(x)$")
plt.grid()
plt.ylabel(r"$y$")
plt.xlabel(r"$x$")
plt.legend()
plt.show()

# The results are similar to the ones we obtained at the point b). We have a better
# approximation when $n=15$ than when $n=4$, and the perturbations on the data have
# very little effect on the piecewise interpolating polynomial.

### END SOLUTION

<hr style="clear:both">

### Interpolation with cubic splines

We consider the functions

$$
g_1(x) = \sin(5 x)
$$

and 

$$
g_2(x) = x |x|
$$

on the interval $[-1, 1]$.

<div class="alert alert-success">
    
**Exercise 4 (Theoretical):** Based on the results you have seen in the lecture, what order of convergence $p$ can we expect a cubic spline to have when interpolating the functions $g_1$ and $g_2$ on equidistant nodes?
</div>

=== BEGIN MARK SCHEME ===

Since $g_1$ is an infinitely differentiable function (i.e. in class $C^{\infty}$), we can directly apply Theorem 2.4 to see that the order of convergence is $p = 4$.

On the other hand, $g_2'(x) = 2|x|$ is not differentiable at $x = 0$, and therefore only in class $C^1$. We cannot apply Theorem 2.4 to this case. 

=== END MARK SCHEME ===

<div class="alert alert-success">
    
**Exercise 5:** Use cubic splines to interpolate the functions $g_1$ and $g_2$ at $n + 1$ equidistant nodes for $n = 2^k$ and $k = 4, 5, \dots, 8$ in the interval $[-1, 1]$. For every $h = 2 / n$, approximate the maximum absolute error

$$
E_{3, h} = \max_{x \in [-1, 1]} |g(x) - s_{3, h}(x)|
$$

by evaluating the spline and the function in $100$ equally spaced points within the interval $[-1, 1]$ and computing the maximum absolute distance attained in these points. Plot $E_{3, h}$ against all $h$ for both functions in a log-log plot (use `plt.loglog` for this). Determine the order of convergence to each of the functions by adding the lines $(h, h^p)$ for some $p=1, 2, \dots$ to your plot. Are your results in accordance with Exercise 4?

*Hint:* The SciPy function `spline = sp.interpolate.CubicSpline(x, y)` interpolates some data `x` and `y` with a cubic spline, and returns a a function `spline`, which can be called to evaluate the spline at some points. 
</div>

In [ ]:
### BEGIN SOLUTION
def g_1(x):
    return np.sin(5 * x)

def g_2(x):
    return x * np.abs(x)

x_lin = np.linspace(-1, 1, 100)  # fine grid
errors_g_1 = []
errors_g_2 = []
n_list = 2 ** np.arange(4, 9)
h_list = 2 / n_list
for n in n_list:
    x_interp = np.linspace(-1, 1, n + 1)  # interpolation nodes
    y_g_1 = g_1(x_interp) 
    y_g_2 = g_2(x_interp)
    s3h_g_1 = sp.interpolate.CubicSpline(x_interp, y_g_1)
    s3h_g_2 = sp.interpolate.CubicSpline(x_interp, y_g_2)
    errors_g_1.append(np.max(np.abs(g_1(x_lin) - s3h_g_1(x_lin))))
    errors_g_2.append(np.max(np.abs(g_2(x_lin) - s3h_g_2(x_lin))))

plt.loglog(h_list, errors_g_1, label=r"error $E_{3, h}$ for $g_1$")
plt.loglog(h_list, np.array(h_list)**5, label=r"$\mathcal{O}(h^5)$", linestyle="--", color="tab:blue")
plt.loglog(h_list, errors_g_2, label=r"error $E_{3, h}$ for $g_2$")
plt.loglog(h_list, np.array(h_list)**2, label=r"$\mathcal{O}(h^2)$", linestyle="--", color="tab:orange")
plt.xlabel(r"$h$")
plt.grid()
plt.legend()
plt.show()

# The maximum absolute interpolation error of the interpolating spline for $g_1$
# goes down approximately as the line $h^5$. Therefore, we observe an even faster
# convergence than what is guaranteed by Theorem 2.4. 
# For $g_2$ the error goes down with $h^2$. As expected, the bound from Theorem 2.4
# does not hold in this case.

### END SOLUTION

<hr style="clear:both">

### Least squares fitting

Let us consider the function

$$  h(x) = \frac{1}{2+x}, \quad x\in [-1, 1], $$

and the points $x_1=-1$, $x_2=0$ et $x_3 =1$. The polynomial $p(x) = a_0 + a_1 x$ which minimizes the function

$$
\Phi(a_0, a_1) = \sum_{i=1}^3 [h(x_i) - p(x_i)]^2= \sum_{i=1}^3 [h(x_i) - a_0 - a_1 x_i]^2
$$

is the polynomial of least squares error of degree 1, also referred to as the regression line.

<div class="alert alert-success">
    
**Exercise 6 (Theoretical):** Write and solve the system of equations to compute the coefficients $a_0$ and $a_1$ of the least squares polynomial $p(x) = a_0 + a_1 x$ for the data points $(x_i,h(x_i))$, $i=1,2,3$.
</div>

=== BEGIN MARK SCHEME ===

The system of equations is given by

$$
V^\top V\mathbf{a}=V^\top\mathbf{y}
$$

where $\mathbf{a}=[a_0 \,\, a_1]^\top$ is the unknowns vector, $\mathbf{y}=[h(x_1) \,\, h(x_2) \,\,h(x_3) ]^\top$ is the data vector and $V$
is the Vandermonde's matrix

$$
V = 
\begin{bmatrix}
1 & x_1 \\ 
1 & x_2 \\
1 & x_3
\end{bmatrix}
=
\begin{bmatrix}
1 & -1 \\ 
1 & 0 \\
1 & 1
\end{bmatrix}.
$$

We have

$$
V^\top V = 
\begin{bmatrix} 
  1 & 1 & 1 \\ 
  -1 & 0 & 1
\end{bmatrix}
\begin{bmatrix} 
  1 & -1 \\ 
  1 & 0 \\ 
  1 & 1
\end{bmatrix}
=
\begin{bmatrix} 
  3 & 0 \\ 
  0 & 2
\end{bmatrix}
$$

and

$$
V^\top y=
\begin{bmatrix} 
  1 & 1 & 1 \\ 
  -1 & 0 & 1 
\end{bmatrix}
\begin{bmatrix} 
  1 \\ 
  1/2 \\ 
  1/3 
\end{bmatrix}
=
\begin{bmatrix} 
  11/6 \\ 
  -2/3
\end{bmatrix}.
$$

The $2\times 2$ system to solve is therefore

$$
\begin{bmatrix} 
  3 & 0 \\ 
  0 & 2
\end{bmatrix}
\begin{bmatrix} 
  a_0 \\ 
  a_1
\end{bmatrix} =
\begin{bmatrix} 
  11/6 \\ 
  -2/3
\end{bmatrix},
$$

whose solution is

$$
a_0 = \frac{11}{18}\quad \text{and} \quad a_1 = -\frac{1}{3}.
$$


=== END MARK SCHEME ===

<div class="alert alert-success">
    
**Exercise 7:** Visualize the function $h$ and the least squares polynomial $p$ you have found in Exercise 6 by plotting their values at $100$ uniformly spaced points on the interval $[-1, 1]$.
</div>

In [ ]:
### BEGIN SOLUTION

def h(x):
    return 1 / (2 + x)

def p(x):
    return 11 / 18 - 1 / 3 * x

x_lin = np.linspace(-1, 1)
plt.plot(x_lin, h(x_lin), label=r"$h(x)$")
plt.plot(x_lin, p(x_lin), label=r"$p(x)$")
plt.xlabel(r"$x$")
plt.legend()
plt.grid()
plt.show()

### END SOLUTION

<hr style="clear:both">

## The end

Splendid! You have reached the end of the fifth exercise notebook. Almost halfway :)